In [25]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [26]:
import pandas as pd
import math
import keras
from keras.layers import Dense,Embedding, Flatten, Conv1D, GlobalMaxPooling1D, LSTM, Bidirectional, Dropout
from keras.preprocessing.text import text_to_word_sequence,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from new_train_val_test import cc_train_val_test

In [27]:
train,val,test,tokenizer = cc_train_val_test.get_upsampled_dataset()

In [28]:
train_x,train_y = train
val_x,val_y = val
test_x,test_y = test

In [29]:
filepath="./models/1xBilstm-{epoch:02d}-{loss:.2f}-{categorical_accuracy:.2f}-{val_loss:.2f}-{val_categorical_accuracy:.2f}-singlelabel.hdf5"
saveModelCallBack = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
from keras.callbacks import Callback

class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

callbacks_list = [saveModelCallBack,tbCallBack,TestCallback((test_x, test_y))]


In [30]:
model = keras.Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 300))
model.add(Bidirectional(LSTM(300, dropout=0.5)))
model.add(Dense(12, activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         5017216   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_2 (Dense)              (None, 12)                3084      
Total params: 5,283,468
Trainable params: 5,283,468
Non-trainable params: 0
_________________________________________________________________


In [33]:
train_x.shape

(177792, 15)

In [35]:
train_y

,is_anagram,is_homophone,is_double,is_cryptic,is_contain,is_reverse,is_alternate,is_init,is_delete,is_hidden,is_spoonerism,is_palindrome
28508,True,False,False,False,False,False,False,False,False,False,False,False
28315,True,False,False,False,False,False,False,False,False,False,False,False
9748,True,False,False,False,False,False,False,False,False,False,False,False
40235,True,False,False,False,False,False,False,False,False,False,False,False
77390,True,False,False,False,False,False,False,False,False,False,False,False
42457,True,False,False,False,False,False,False,False,False,False,False,False
22207,True,False,False,False,False,False,False,False,False,False,False,False
47792,True,False,False,False,False,False,False,False,False,False,False,False
59895,True,False,False,False,False,False,False,False,False,False,False,False
39748,True,False,False,False,False,False,False,False,False,False,False,False


In [34]:
train_y.shape

(177792, 12)

In [21]:
test_y.shape

(9404, 12)

In [24]:
history = model.fit(train_x,train_y ,validation_data=(test_x,test_y), batch_size=128, epochs=15,shuffle=True,callbacks=callbacks_list,initial_epoch=1)

Train on 177792 samples, validate on 9404 samples
Epoch 2/15
  1408/177792 [..............................] - ETA: 2:10 - loss: 1.8384 - categorical_accuracy: 0.3097

KeyboardInterrupt: 

In [ ]:
import pickle
with open('history.pickle','wb') as f:
    pickle.dump(history,f)